##Imports

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Load, split and preprocess the dataset

In [0]:
(x_train, y_train), (x_tv, y_tv) = mnist.load_data()
x_val, x_test, y_val, y_test = train_test_split(x_tv, y_tv, 
                                                test_size=0.5, random_state=42)

rows, cols = 28, 28
num_classes = 10

x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
x_val = x_val.reshape(x_val.shape[0], rows, cols, 1)
x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)
input_shape = (rows, cols, 1)

x_train = x_train.astype('float32') / 255
x_val = x_val.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Define F1 score

In [0]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

##Define basic CNN

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', f1])

## Define F1 score

##Training process

In [5]:
early_stopping = EarlyStopping(monitor='val_acc', mode='max', patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('best.model', monitor='val_acc', mode='max', save_best_only=True, verbose=1)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.7, patience=3, verbose=1)

model.fit(x_train, y_train,
          batch_size=512,
          epochs=100,
          callbacks=[early_stopping, model_checkpoint, reduce_lr_on_plateau],
          verbose=1,
          validation_data=(x_val, y_val))

Train on 60000 samples, validate on 5000 samples
Epoch 1/100
60000/60000 [==============================] - 13s 222us/step - loss: 0.8217 - acc: 0.8036 - f1: 0.7955 - val_loss: 0.1547 - val_acc: 0.9506 - val_f1: 0.9509

Epoch 00001: val_acc improved from -inf to 0.95060, saving model to best.model
Epoch 2/100
60000/60000 [==============================] - 9s 158us/step - loss: 0.1395 - acc: 0.9580 - f1: 0.9581 - val_loss: 0.1182 - val_acc: 0.9622 - val_f1: 0.9623

Epoch 00002: val_acc improved from 0.95060 to 0.96220, saving model to best.model
Epoch 3/100
60000/60000 [==============================] - 9s 158us/step - loss: 0.0976 - acc: 0.9707 - f1: 0.9708 - val_loss: 0.1166 - val_acc: 0.9676 - val_f1: 0.9680

Epoch 00003: val_acc improved from 0.96220 to 0.96760, saving model to best.model
Epoch 4/100
60000/60000 [==============================] - 9s 158us/step - loss: 0.0748 - acc: 0.9774 - f1: 0.9777 - val_loss: 0.0318 - val_acc: 0.9908 - val_f1: 0.9910

Epoch 00004: val_acc improv

##Results

In [7]:
model = load_model('best.model', custom_objects={'f1': f1})
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]: 0.5f}')
print(f'Test accuracy:{score[1]: 0.5f}')
print(f'Test f1_score:{score[2]: 0.5f}')

Test loss:  0.01795
Test accuracy: 0.99380
Test f1_score: 0.99380
